In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 画像セグメンテーション (Image segmentation with SAM 3)

このノートブックでは、SAM 3を使用して画像セグメンテーションを行う方法を解説します。
以下の2つの機能について学びます：

- **テキストプロンプト**: 自然言語（例：「person」、「face」）を使用してオブジェクトをセグメントします。
- **ボックスプロンプト**: バウンディングボックスを指定して、その領域内のオブジェクトをセグメントします。

# <a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam3/blob/main/notebooks/sam3_image_predictor_example.ipynb">
#   <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
# </a>

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

import sam3
from PIL import Image
from sam3 import build_sam3_image_model
from sam3.model.box_ops import box_xywh_to_cxcywh
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualization_utils import draw_box_on_image, normalize_bbox, plot_results

sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")

In [ ]:
import torch

# Ampere GPU向けにtfloat32を有効化
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# ノートブック全体でbfloat16を使用
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

# モデル構築 (Build Model)

SAM 3 モデルを構築し、必要なリソース（BPEファイルなど）を読み込みます。

In [ ]:
bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
model = build_sam3_image_model(bpe_path=bpe_path)

画像を読み込み、`Sam3Processor` を初期化します。
`Sam3Processor` は、画像の前処理や推論の実行を管理するクラスです。

In [ ]:
image_path = f"{sam3_root}/assets/images/test_image.jpg"
image = Image.open(image_path)
width, height = image.size
processor = Sam3Processor(model, confidence_threshold=0.5)
inference_state = processor.set_image(image)

# テキストプロンプト (Text prompt)

自然言語のテキストプロンプトを使用して、画像内のオブジェクトをセグメントします。
ここでは "person"（人）というテキストを入力として与えます。

In [ ]:
input_text = "person"
masks, scores, boxes, object_ids = processor.predict_text(input_text, inference_state)
print(f"found {len(masks)} object(s)")

In [ ]:
plot_results(masks, scores, object_ids, image)

# ボックスプロンプト (Box prompt)

バウンディングボックスを指定して、特定の領域をセグメントします。
ここでは、画像内の特定の人物の顔を囲むボックスを指定します。

In [ ]:
# ボックスは [x, y, w, h] 形式です
input_box = [300, 50, 200, 200]
# ボックスを正規化された [cx, cy, w, h] 形式に変換します
box_cxcywh = normalize_bbox(box_xywh_to_cxcywh(torch.tensor(input_box)), height, width)

masks, scores, boxes, object_ids = processor.predict_box(box_cxcywh, inference_state)
print(f"found {len(masks)} object(s)")

In [ ]:
plot_results(masks, scores, object_ids, image)
draw_box_on_image(image, input_box)